In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
from PIL import Image

# Load the saved model
model = tf.keras.models.load_model('malaria_cnn_model.h5')  # Update with the correct file name


In [3]:
IMG_HEIGHT, IMG_WIDTH = 150, 150

# Function to preprocess the image
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
    img_array = img_to_array(img) / 255.0  # Normalize pixel values
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Function to predict and display results
def predict_image(image_path):
    # Preprocess image
    img_array = preprocess_image(image_path)
    
    # Perform prediction
    prediction = model.predict(img_array)
    
    # Extract the predicted class and confidence
    predicted_class = 'Parasite' if prediction[0] < 0.5 else 'Uninfected'
    confidence = prediction[0][0] if prediction[0] > 0.5 else 1 - prediction[0][0]
    
    # Display the image
    img = Image.open(image_path)
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Prediction: {predicted_class}\nConfidence: {confidence:.2f}")
    plt.show()

# Create file upload widget
uploader = widgets.FileUpload(
    accept='.jpg,.png,.jpeg',  # Accept only image files
    multiple=False  # Allow only single file upload
)

# Button to trigger prediction
button = widgets.Button(description="Predict")

# Output area to display results
output = widgets.Output()

# Event handler for button click
def on_button_click(b):
    with output:
        clear_output(wait=True)  # Clear previous output
        if uploader.value:
            # Get the uploaded image file (handling tuple correctly)
            uploaded_file = uploader.value[0] 
            content = uploaded_file['content']            
            
            # Save the uploaded image to disk
            file_path = 'uploaded_image.jpg'
            with open(file_path, 'wb') as f:
                f.write(content)
            
            # Predict and display results
            predict_image(file_path)
        else:
            print("Please upload an image.")

# Link the button click event to the handler
button.on_click(on_button_click)

# Display widgets and output
display(widgets.VBox([uploader, button, output]))